In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.model_selection import cross_val_score,StratifiedKFold
from sklearn.metrics import roc_auc_score,mean_squared_error
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [2]:
X_train=pd.read_csv('train_offline.csv')
X_test=pd.read_csv('test_offline.csv')
all_data=pd.concat([X_train,X_test],axis=0)
train_num=X_train.shape[0]

C:\Users\allen\Anaconda3\envs\tensorflow-GPU\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
all_data

,Coupon_id,Date,Date_received,Discount_rate,Distance,Merchant_id,User_id
0,NaN,20160217.0,NaN,NaN,0.0,2632,1439408
1,8591.0,NaN,20160217.0,20:1,0.0,2632,1439408
2,1078.0,NaN,20160319.0,20:1,0.0,2632,1439408
3,7610.0,NaN,20160429.0,200:20,0.0,3381,1832624
4,11951.0,NaN,20160129.0,200:20,1.0,3381,2029232
5,9776.0,NaN,20160129.0,10:5,2.0,3381,2223968
6,12034.0,NaN,20160207.0,100:10,NaN,2099,73611
7,5054.0,NaN,20160421.0,200:30,10.0,1569,163606
8,7802.0,NaN,20160130.0,200:20,10.0,4833,3273056
9,7610.0,NaN,20160412.0,200:20,2.0,3381,94107


In [ ]:
X_train['DateTime_received'] = X_train['Date_received'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
X_train['DateTime'] = X_train['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
X_test['DateTime_received'] = X_test['Date_received'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
X_test=X_test[~(X_test['DateTime_received'].isnull())]  #也可用 X_test.dropna(subset=['DateTime_received'])

In [ ]:
label=[]
x=np.datetime64('NaT')
for r,u in zip(X_train['DateTime_received'].values,X_train['DateTime'].values) :
    if r==x:
        label.append('Invalid')
    elif r!=x and u==x:
        label.append('F')
    elif r!=x and u!=x:
        if u-r<=np.timedelta64(15,'D'):
            label.append('T')
        else :
            label.append('F')
            
X_train['Label']=label
X_train=X_train[~(all_data['Label']=='Invalid')]
y_train=X_train['Label']


In [ ]:
X_train['received_year'] =X_train['DateTime_received'].apply(lambda x: datetime.datetime.strftime(x, '%Y')).astype('int64')
X_train['received_month'] =X_train['DateTime_received'].apply(lambda x: datetime.datetime.strftime(x, '%m')).astype('int64')
X_train['received_day'] =X_train['DateTime_received'].apply(lambda x: datetime.datetime.strftime(x, '%d')).astype('int64')


In [ ]:
X_test['received_year'] =X_test['DateTime_received'].apply(lambda x: datetime.datetime.strftime(x, '%Y')).astype('int64')
X_test['received_month'] =X_test['DateTime_received'].apply(lambda x: datetime.datetime.strftime(x, '%m')).astype('int64')
X_test['received_day'] =X_test['DateTime_received'].apply(lambda x: datetime.datetime.strftime(x, '%d')).astype('int64')



In [ ]:
X_train.drop(['Date','Date_received','DateTime','DateTime_received','Label'],axis=1,inplace=True)
X_test.drop(['Date_received','DateTime_received'],axis=1,inplace=True)